In [1]:
import os
import subprocess as sp
from iterextras import par_for

In [2]:
CRATES = [
    ('Rocket', 'core/lib/', None),
    ('image', '', None),
    ('rayon', '', None),
    ('rg3d', '', None),    
    #('RustPython/vm', ''),
    ('diesel', 'diesel/', None),
    ('nalgebra', '', None),
    # ('rav1e', ''),
    ('rustls', 'rustls/', None),
    ('sccache', '', ''),
    ('hyper', '', '--features full'),    
]

In [3]:
def analyze_crate(crate):
    (name, subdir, feature_flags) = crate
    lib_path = os.path.abspath(f'../data/repos/{name}/{subdir}src/lib.rs')
    out_path = os.path.expanduser(f'~/Code/rust-slicer/data/{name.replace("/", "_")}.json')
    
    if os.path.exists(out_path) and os.stat(out_path).st_size > 0:
        return
    
    dyld_lib_path = 'DYLD_LIBRARY_PATH=$($(rustup which --toolchain local rustc) --print target-libdir):$DYLD_LIBRARY_PATH'
    feature_flags = feature_flags if feature_flags is not None else "--all-features"
    cargo_cmd = f'cargo check {feature_flags}'
    cmd = f'rm -f {out_path} && {cargo_cmd} && {dyld_lib_path} RUST_BACKTRACE=1 RUST_LOG="rust_slicer_eval=debug" rust-slicer-eval {lib_path} {feature_flags} > {out_path}'
    try:
        # print(cmd)
        sp.check_output(cmd, cwd=f'../data/repos/{name}', shell=True, stderr=sp.PIPE)
    except sp.CalledProcessError as e:
        print(f'Stderr for {name}')
        print(e.stderr.decode('utf-8'))

In [ ]:
par_for(analyze_crate, CRATES)

  0%|          | 0/9 [00:00<?, ?it/s]